In [ ]:
# for debugging the jcapper loader code, run this script "python -i load_jcapper.py"
import sys
sys.path.append('I:/YaoTony/code/x8313')
sys.path.append('/Users/saleemhuda/Dropbox/X8313_Live')
from datetime import date
import pytest
from pandas import date_range
import pandas as pd

def compute_simple_payout(df, attr_model, ascending=False, bet_amount=1.0):
    """
    Add columns for quick calculation of Win bets % payout
    :param df: Dataframe from dataset for multiple races
    :param attr_model: (string)an attribute / prob or score in the dataframe that can be ranked
    :param ascending: False if best is higher score i.e. probs
    :param bet_amount: default to 1.0
    :return: Dataframe with columns added

    """
    df['is_win'] = df['official_finish_position'].map(lambda x:int(x==1))
    #df['rank_' + attr_model] = df.groupby('race_id')[attr_model].transform(lambda x:x.rank(ascending=False))
    df['bet_amount'] = 1.0#df['rank_' + attr_model].map(lambda x:int(x<1.5)*bet_amount)
    df['is_wager'] = df['bet_amount'].map(lambda x:int(x>0))
    df['is_paid'] = df['is_wager'] * df['is_win']
    df['payout'] = df['is_win']*df['bet_amount']*df['payout_win'].fillna(0.0)

    return df

def add_criteria(df, criteria_list):
    print("num_runners: {}".format(len(df.runner_id.unique())))

    for c in criteria_list:
        df = df[c]
        print("num_runners: {}".format(len(df.runner_id.unique())))
    return df
    

def compute_advantage(df,rebate=0.0):

    pct_win = df.groupby('race_id')['is_paid'].sum().value_counts(normalize=True)[1]

    pct_loss = 1.0 - pct_win
    mean_odds = df[df.is_paid>0]['final_tote_odds'].mean()
    advantage = pct_win  - pct_loss / mean_odds
    print(advantage)
    return advantage


In [ ]:
df_hist = pd.read_csv('./dfX_hist.csv')
df_hist = df_hist[df_hist['track_id']=='LAD']
df_hist['payout_win'] = df_hist['final_tote_odds']+1

In [ ]:
attr = 'prob_x8runner_HDWPSRRating_norm_par'
rbt = 0.0865
#since advantage is nothing about bet_amount, we use the original functions without bet_amount type.
df_1 = compute_simple_payout(df_hist, attr_model = attr, ascending=False, bet_amount=1.0)
advantage = compute_advantage(df_1,rebate=rbt)

In [ ]:
crit_par_HDW = df_hist.x8is_HDWPSRRating_norm_par>0
crit_not_top_HDW = df_hist.x8runner_rank_HDWPSRRating_norm_par>1
criteria_list_speed = [crit_par_HDW, crit_not_top_speed, crit_not_top_HDW]

In [ ]:
df_hist = add_criteria(df=df_hist, criteria_list=criteria_list_speed)

In [ ]:
df_hist

In [ ]:
compute_advantage(df_hist)

In [ ]:
df_hist_filter.prob_x8runner_HDWPSRRating_norm_par
